# Lecture 9: graphs!

In this notebook we will implement a few classes (CS161Vertex, CS161Graph) that we'll use repeatedly.

We'll also implement BFS and DFS, alsong with a few applications

In [1]:
# First, Vertex and Graph classes for directed graphs

class CS161Vertex:
    def __init__(self, v):
        self.inNeighbors = []
        self.outNeighbors = []
        self.value = v
        # useful for DFS
        self.inTime = None
        self.outTime = None
        self.status = "unvisited"
        
    def hasOutNeighbor(self,v):
        if v in self.outNeighbors:
            return True
        return False
        
    def hasInNeighbor(self,v):
        if v in self.inNeighbors:
            return True
        return False
    
    def hasNeighbor(self,v):
        if v in self.inNeighbors or v in self.outNeighbors:
            return True
        return False
    
    def getOutNeighbors(self):
        return self.outNeighbors
    
    def getInNeighbors(self):
        return self.inNeighbors
        
    def addOutNeighbor(self,v):
        self.outNeighbors.append(v)
    
    def addInNeighbor(self,v):
        self.inNeighbors.append(v)
    
    def __str__(self):
        return str(self.value) 
        
# This is a directed graph class for use in CS161.
# It can also be used as an undirected graph by adding edges in both directions.
class CS161Graph:
    def __init__(self):
        self.vertices = []

    def addVertex(self,n):
        self.vertices.append(n)
        
    # add a directed edge from CS161Node u to CS161Node v
    def addDiEdge(self,u,v):
        u.addOutNeighbor(v)
        v.addInNeighbor(u)
        
    # add edges in both directions between u and v
    def addBiEdge(self,u,v):
        self.addDiEdge(u,v)
        self.addDiEdge(v,u)
        
    # get a list of all the directed edges
    # directed edges are a list of two vertices
    def getDirEdges(self):
        ret = []
        for v in self.vertices:
            ret += [ [v, u] for u in v.outNeighbors ]
        return ret
    
    def __str__(self):
        ret = "CS161Graph with:\n"
        ret += "\t Vertices:\n\t"
        for v in self.vertices:
            ret += str(v) + ","
        ret += "\n"
        ret += "\t Edges:\n\t"
        for a,b in self.getDirEdges():
            ret += "(" + str(a) + "," + str(b) + ") "
        ret += "\n"
        return ret

In [2]:
# Quick sanity-check
G = CS161Graph()
for i in range(10):
    G.addVertex( CS161Vertex(i) )
V = G.vertices
for i in range(9):
    G.addDiEdge( V[i], V[i+1] )
print(G)

CS161Graph with:
	 Vertices:
	0,1,2,3,4,5,6,7,8,9,
	 Edges:
	(0,1) (1,2) (2,3) (3,4) (4,5) (5,6) (6,7) (7,8) (8,9) 



## Depth-first search

In [3]:
def DFS_helper( w, currentTime, verbose ):
    if verbose:
        print("Time", currentTime, ":\t entering", w)
    w.inTime = currentTime
    currentTime += 1
    w.status = "inprogress"
    for v in w.getOutNeighbors():
        if v.status == "unvisited":
            currentTime = DFS_helper(v, currentTime, verbose)
            currentTime += 1
    w.outTime = currentTime
    w.status = "done"
    if verbose:
        print("Time", currentTime, ":\t leaving", w)
    return currentTime
        
def DFS( w, G, verbose=False ):
    for v in G.vertices:
        v.status = "unvisited"
        v.inTime = None
        v.outTime = None
    return DFS_helper( w, 0, verbose )

### Application: Topological sorting

Now we can use DFS to topologically sort!

Below, we modify our DFS algorithm (and call it "topoSort") to additionally keep track of a list of vertices.

In [13]:
# This method is not complete!
# It will only topologically sort the vertices reachable from the first node that it picks.
# How would you modify it to be complete?


def topoSort_helper( w, currentTime, ordering, verbose ):
    if verbose:
        print("Time", currentTime, ":\t entering", w)
    w.inTime = currentTime
    currentTime += 1
    w.status = "inprogress"
    for v in w.getOutNeighbors():
        if v.status == "unvisited":
            currentTime = topoSort_helper(v, currentTime, ordering, verbose)
            currentTime += 1
    w.outTime = currentTime
    w.status = "done"
    ordering.insert(0, w)
    if verbose:
        print("Time", currentTime, ":\t leaving", w)
    return currentTime
        
def topoSort( G, verbose=False ):
    w = G.vertices[0]
    ordering = []
    for v in G.vertices:
        v.status = "unvisited"
        v.inTime = None
        v.outTime = None
    topoSort_helper( w, 0, ordering, verbose )
    return ordering

Let's test this on the example on the slides in class

In [14]:
G = CS161Graph()
dpkg = CS161Vertex( "dkpg" )
coreutils = CS161Vertex( "coreutils" )
multiarch_support = CS161Vertex( "multiarch_support" )
libselinux1 = CS161Vertex("libselinux1")
libbz2 = CS161Vertex( "libbz2" )
tar = CS161Vertex("tar")
for N in [ dpkg, coreutils, multiarch_support, libselinux1, libbz2, tar ]:
    G.addVertex( N )
G.addDiEdge( dpkg, multiarch_support )
G.addDiEdge( dpkg, coreutils )
G.addDiEdge( dpkg, tar )
G.addDiEdge( dpkg, libbz2 )
G.addDiEdge( coreutils, libbz2 )
G.addDiEdge( coreutils, libselinux1 )
G.addDiEdge( libbz2, libselinux1 )
G.addDiEdge( libselinux1, multiarch_support )

In [15]:
print(G)

CS161Graph with:
	 Vertices:
	dkpg,coreutils,multiarch_support,libselinux1,libbz2,tar,
	 Edges:
	(dkpg,multiarch_support) (dkpg,coreutils) (dkpg,tar) (dkpg,libbz2) (coreutils,libbz2) (coreutils,libselinux1) (libselinux1,multiarch_support) (libbz2,libselinux1) 



In [16]:
V = topoSort(G)
for v in V:
    print(v)

dkpg
tar
coreutils
libbz2
libselinux1
multiarch_support


## BFS

In [ ]:
def BFS(w, G):
    for v in G.vertices:
        v.status = "unvisited"
    n = len(G.vertices)
    Ls = [ [] for i in range(n) ]
    Ls[0] = [w]
    w.status = "visited"
    for i in range(n):
        for u in Ls[i]:
            for v in u.getOutNeighbors():
                if v.status == "unvisited":
                    v.status = "visited"
                    Ls[i+1].append(v)
    return Ls

### Application: distance

In [ ]:
# This is the graph that was on the slides in class (vertex 1 was Kevin Bacon)
G = CS161Graph()
for i in range(7):
    G.addVertex( CS161Vertex( i ) ) 
V = G.vertices
for pairs in [ (0,2), (0,4), (0, 6), (1, 3), (1, 4), (2, 3), (2, 6), (4, 5), (5,6) ]:
    G.addBiEdge( V[pairs[0]], V[pairs[1]] )
print(G)

In [ ]:
levels = BFS(G.vertices[1], G)
for i in range(len(levels)):
    print("Level", i, ":")
    for j in levels[i]:
        print("\t", j)
    
# now what's the distance from node 0 to node 6?
for i in range(len(levels)):
    if G.vertices[6] in levels[i]:
        print("distance from 1 to 6 is", i)
        break